The goal of this notebook is to write python code to aproximate the price of an ATMF straddle. An ATMF straddle (or At-The-Money Forward straddle) is a type of investment strategy used primarily in options trading. 

An option is considered "ATM" (or At-The-Money) when the strike price of the option is equal to the current price of the underlying asset. A straddle involves buying a call option and a put option with the same strike price/same expiration date. The idea behind a straddle is to profit from significant price movements on the underlying asset regardless of direction. The Forward element of this ATMF straddle dictates that the strike price is set at the forward price expected at the expiration of the option. The forward price is typically calculated based on the current prices plus any carry costs (mainly interests or dividends) until the option's maturity.

From JPMorgan Python training course, here is the formula to approximate the price of an ATMF straddle:


$$STRADDLE_ATMF=(2/sqrt(2*pi))*F*rho*sqrt(T)$$

with:

$$rho=implied volatily$$
$$T=time to maturity$$
$$F=forward of the underlier$$
$$pi=pi (of course)$$

Components:
- 1/sqrt(2*pi): This component is a part of the normalization factor used in the probability density function of a standard normal distribution, often used in formulas derived from the Black-Scholes model.
- vol(rho): Implied volatility represents the expected fluctuation in the underlying asset's price. Higher volatility increases the premium of both call and put options because the expected movement of the stock price away from the strike price increases.
- sqrt(time): The square root of time reflects how option prices are sensitive not just to the length of time to expiration but to the square root of this time, which aligns with the principles of stochastic processes where variance of stock price returns increases with time.

This formula effectively calculates the price of an ATMF straddle by estimating the expected magnitude of the underlying asset's price movement whithin the given time frame, under the assumption of a log-normal distribution of price changes (a common assumption in financial models).
- 2x: This represents buying both a call and a put. Since we are buying both a call and a put where both options are at-the-money, their premiums can be expected to be similar, hence the factor of 2.
- Volatility and Time: The product of volatility and the square root of time gives a measure of the expected range (standard deviation) of the price movement. Multiplying this by the normalization constant of the normal distribution adjusts this range into a probability-weighted value.

Lets start by defining these variables and create the formula and calling it.



In [16]:
vol=0.2 #this is rho, implied volatiliy
time=1. #time to maturity, adding the "." to make it a float instead of int

#define a function called straddlePricer to be called whenever necessary. Note, **=sqrt
def straddlePricer(vol,time):
    return 2. * ((1./(2*3.14) ** 0.5) * vol * time ** 0.5)

print(straddlePricer(vol,time))

0.15961737689352445


This result can be interpreted as the cost of buying both the call and the put option at-the-money forward, as a fraction of the underlying asset's price. In other words, it's bassically 15.93% of the current price of the underlying asset. Let's try making the inputs optional.

In [17]:
def straddlePricer(vol=0.2,time=1.0):
    return 2. * ((1./(2*3.14) ** 0.5) * vol * time ** 0.5)

print(straddlePricer())
print(straddlePricer(0.22))
print(straddlePricer(0.22, 2.0))

0.15961737689352445
0.1755791145828769
0.24830636511256418


Next step, lets try getting rid of the approximation of pi and the **0.5 for sqrt in my function using numpy as np.

In [18]:
import numpy as np
def straddlePricer(vol=0.2,time=1.0):
    return 2. * ((1./np.sqrt(2*np.pi)) * vol * np.sqrt(time))

print(straddlePricer())


0.1595769121605731



This is where we start updating the notebook to make it better. We will create a Monte Carlo simulation.

The Monte Carlo method is a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 

I won't go into more details of Monte Carlo simulations (see my Physics/Engineering repositories to see where I've learned and applied about Monte Carlo). Monte Carlo is very popular in finance such as risk assessment and modeling financial instruments like options and derivatives.

To create the Monte Carlo simulations, we need to create some variance. Variance scales linearly with time. We can either:

1. divide the variance by time and take the square root to get a daily volatility, or
2. take the square root of variance (volatility) and divide by the root of time.

Generally, the latter is clearer and simpler to understand since we typically think in vol terms, but lets compare both methods for fun.



In [19]:
## daily Vol here
vol=0.2
var=vol**2
sqrtVarOverTime=np.sqrt(var/252)
volOverSqrtTime=vol/np.sqrt(252)
valuesEqual=np.isclose(sqrtVarOverTime, volOverSqrtTime)
print(f'sqrtVarOverTime = {sqrtVarOverTime}\nvolOverSqrtTime={volOverSqrtTime}\nAre they close? {valuesEqual}')


sqrtVarOverTime = 0.012598815766974242
volOverSqrtTime=0.01259881576697424
Are they close? True


Using volOverSqrtTime, we want to generate multiple days of return. np.random.normal takes three optional inputs: mean, standard deviation, and size of the array. 

Lets generate 252 random numbers, representing a full year of returns. Mean will be set at 0.

In [20]:
time=1 #time in years
nDays=time*252  #252 trading days in a year times the number of years chosen
dailyVol=vol/np.sqrt(252.)
print(nDays)

np.random.normal(0, dailyVol, nDays)

252


array([-2.42426747e-03, -3.54659093e-03, -8.95127102e-03,  1.88575908e-02,
        4.01804212e-03, -6.45142708e-03,  5.70865292e-03,  1.37676686e-02,
        2.14072491e-02,  7.50493197e-03,  8.13083365e-03,  2.16504183e-02,
        9.00990176e-03,  1.72390962e-02, -3.99464225e-03, -6.15059403e-03,
       -1.21555224e-02,  1.54301093e-02,  6.65568977e-03, -2.50727812e-03,
       -4.66510079e-03, -4.17057130e-03, -1.14366344e-02,  2.53343689e-02,
        3.47274279e-03,  2.64150194e-02,  1.64851234e-03,  7.91317335e-03,
        2.53249252e-02, -1.32869707e-02,  4.55784991e-03,  2.13381987e-03,
       -1.16669187e-02,  5.28010935e-03, -1.25491220e-02, -1.69617408e-02,
        1.50598506e-02, -1.31208181e-03,  2.33566991e-02,  7.15659927e-03,
        8.39307909e-03, -5.22550220e-03, -1.34360252e-02, -5.60204408e-04,
       -2.33587846e-03, -5.56635102e-03,  1.48196666e-02, -4.64048738e-03,
        8.54148062e-03, -9.03374300e-03, -3.50748884e-03, -4.79073786e-03,
       -1.81752749e-03,  

Using np.random.normal, we will start building the core of our MC simulation.

In [21]:
resultSum=np.abs(np.prod(1+(np.random.normal(0, dailyVol, int(round(time*252)))))-1)
print(resultSum)

0.08419026706340804


We need a for loop that will iterate an "MCPaths" amount of time and take the average of resultSum over the number of MCPaths. Lets set the number of mcPaths to 100 and compare to the straddlePricer() function.

In [22]:
def straddlePricerMC(vol=0.2,time=1.0, mcPaths=100):
    dailyVol=vol/np.sqrt(252.) 
    resultSum=0
    for _ in range(mcPaths):
        resultSum+=np.abs(np.prod((1+np.random.normal(0, dailyVol, int(round(time*252)))))-1)
    return resultSum/mcPaths

#for fun, lets compare the result of both functions
print(straddlePricer())
print(straddlePricerMC())

0.1595769121605731
0.15446790283659495
